In [1]:
import sys 
import os

In [2]:
src_path = os.path.abspath("../")
if src_path not in sys.path:
    sys.path.append(src_path)

In [3]:
from src.tools.md_class_functions import *
from src.tools.md_class_utility import*
from src.tools.md_class_graphs import *
from src.water_md_class import *

### Set path to your lammps file and initialise trajectory object

In [ ]:
## path towards an ion trajectory with recombination happening
water_path = "../testing/recombination_tester.lammpstrj"

In [ ]:
#initialise trajectory object by passing path, format and scaling = 0 means not scaled -> will be scaled in __init__
trj = Trajectory(water_path, format="lammpstrj", scaled=0)

### Access class atributes of the Trajectory object 

In [ ]:
    #access class atributes which are calculated on initialisation, basic information about the trajectory 
    
    print("Trajectory first 3 rows")
    print(trj.trajectory[0, :3, :])
    print("box dimensions")
    print(trj.box_dim[0])
    print("box size")
    print(trj.box_size[0])
    print("number of atoms")
    print(trj.n_atoms)
    print("number of timesteps")
    print(trj.n_snapshots)
    print("species split")
    print("s1 = Hydrogen")
    print(trj.s1[0][:3, :])
    print("s2 = Oxygen")
    print(trj.s2[0][:3, :])
    print("did recombine?")
    print(trj.did_recombine)
    print("Recombination Time")
    print(trj.recombination_time)

### Class Methods


In [ ]:
trj.get_ion_distance()

In [ ]:
plot_ion_distance_euc(trj, (11, 6))

In [ ]:
time_step = 773

In [ ]:
bonding_list, unique_oxygens, ions = trj.get_hydrogen_bonds(timestep=time_step, cutoff=2.9, starting_oh=True)

In [ ]:
plot_hbonds_single(bonding_list, trj.s2[time_step], start="OH", fig_size=(10, 8))

In [ ]:
bonds_H3O, oxygens_H3O, ions =  trj.get_hydrogen_bonds(timestep=time_step, cutoff=2.9, starting_oh=False)
bonds_OH, oxygens_OH, _ =  trj.get_hydrogen_bonds(timestep=time_step, cutoff=2.9, starting_oh=True)

In [ ]:
plot_hbond_network(bonds_OH, bonds_H3O, trj.s2[time_step], ions, fig_size=(10, 8))

In [ ]:
hb_timeseries = get_HB_timeseries(trj)

In [ ]:
plot_HB_network(hb_timeseries, trj.s2, plot_oxygen=True)

In [ ]:
plot_HB_ratio(hb_timeseries, n_atoms=trj.n_atoms)

In [ ]:
wire_lengths, h_bonds = get_all_wires(trj)

In [ ]:
%matplotlib widget

In [ ]:
plot_HB_wire(h_bonds, trj, plot_hydrogens=True)

In [ ]:
plot_wire_length(wire_lengths)

In [ ]:
OO_rdf = trj.get_rdf_rdist()

In [ ]:
HH_rdf = trj.get_rdf_rdist(gr_type="HH")

In [ ]:
plot_rdf(OO_rdf[0], OO_rdf[1])

In [ ]:
plot_rdf(HH_rdf[0], HH_rdf[1], type="HH")

In [ ]:
MSD = trj.get_MSD()

In [ ]:
plot_MSD(MSD)

In [ ]:
diff = trj.get_translational_diffusion(MSD)

### Trajectory Manipulation




In [4]:
#trjwater.lammpstrj is just a water sim trajectory without ions
path_water = "../testing/trjwater.lammpstrj"

In [5]:
traj_2 = Trajectory(file=path_water, format="lammpstrj", scaled=1, verbosity="loud")

Processing Snapshot:1
Processing Snapshot:2
Processing Snapshot:3
Processing Snapshot:4
Processing Snapshot:5
Processing Snapshot:6
Processing Snapshot:7
Processing Snapshot:8
Processing Snapshot:9
Processing Snapshot:10
Processing Snapshot:11
Processing Snapshot:12
Processing Snapshot:13
Processing Snapshot:14
Processing Snapshot:15
Processing Snapshot:16
Processing Snapshot:17
Processing Snapshot:18
Processing Snapshot:19
Processing Snapshot:20
Processing Snapshot:21
Processing Snapshot:22
Processing Snapshot:23
Processing Snapshot:24
Processing Snapshot:25
Processing Snapshot:26
Processing Snapshot:27
Processing Snapshot:28
Processing Snapshot:29
Processing Snapshot:30
Processing Snapshot:31
Processing Snapshot:32
Processing Snapshot:33
Processing Snapshot:34
Processing Snapshot:35
Processing Snapshot:36
Processing Snapshot:37
Processing Snapshot:38
Processing Snapshot:39
Processing Snapshot:40
Processing Snapshot:41
Processing Snapshot:42
Processing Snapshot:43
Processing Snapshot:

Attribute Error occurred(received list instead of numpy array) using 78 element of the list
Attribute Error occurred(received list instead of numpy array) using 79 element of the list
Attribute Error occurred(received list instead of numpy array) using 80 element of the list
Attribute Error occurred(received list instead of numpy array) using 81 element of the list
Attribute Error occurred(received list instead of numpy array) using 82 element of the list
Attribute Error occurred(received list instead of numpy array) using 83 element of the list
Attribute Error occurred(received list instead of numpy array) using 84 element of the list
Attribute Error occurred(received list instead of numpy array) using 85 element of the list
Attribute Error occurred(received list instead of numpy array) using 86 element of the list
Attribute Error occurred(received list instead of numpy array) using 87 element of the list
Attribute Error occurred(received list instead of numpy array) using 88 element 

In [6]:
# generate ion trajectories for ion MD runs
traj_2.get_displace(snapshot=50, distance=0.4, eps=0.05, path="../tutorial_notebook/", num_traj=2)

0
distance too far, trying next O
distance too far, trying next O
distance too far, trying next O
distance too far, trying next O
distance too far, trying next O
distance too far, trying next O
distance too far, trying next O
distance too far, trying next O
distance too far, trying next O
distance too far, trying next O
distance too far, trying next O
distance too far, trying next O
distance too far, trying next O
distance too far, trying next O
distance too far, trying next O
distance too far, trying next O
distance too far, trying next O
distance too far, trying next O
distance too far, trying next O
distance too far, trying next O
distance too far, trying next O
distance too far, trying next O
distance too far, trying next O
distance too far, trying next O
distance too far, trying next O
distance too far, trying next O
distance too far, trying next O
distance too far, trying next O
distance too far, trying next O
distance too far, trying next O
distance too far, trying next O
distan